# RioXarray raster index examples

In [1]:
import xarray as xr

xr.set_options(display_expand_indexes=True)

In [2]:
# TODO: rasterix pyproject
import sys

sys.path.append("..")
from rasterix import RasterIndex

## Example with rectilinear and no rotation affine transform

Both x and y coordinates are 1-dimensional.

In [3]:
source = "/vsicurl/https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/2024/01_Jan/S_20240101_concentration_v3.0.tif"

### Load and inspect the datasets, with and without `RasterIndex`.

In [4]:
da_no_raster_index = xr.open_dataarray(source, engine="rasterio")


def set_raster_index(obj):
    """Return a new DataArray or Dataset with a RasterIndex."""
    x_dim = obj.rio.x_dim
    y_dim = obj.rio.y_dim

    index = RasterIndex.from_transform(
        obj.rio.transform(),
        obj.sizes[x_dim],
        obj.sizes[y_dim],
        x_dim=x_dim,
        y_dim=y_dim,
    )

    # drop-in replacement of explicit x/y coordinates for now
    coords = xr.Coordinates.from_xindex(index)
    return obj.assign_coords(coords)


da_raster_index = set_raster_index(da_no_raster_index)

In [5]:
da_no_raster_index

<xarray.DataArray 'band_data' (band: 1, y: 332, x: 316)> Size: 420kB
[104912 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 3kB -3.938e+06 -3.912e+06 ... 3.912e+06 3.938e+06
  * y            (y) float64 3kB 4.338e+06 4.312e+06 ... -3.912e+06 -3.938e+06
    spatial_ref  int64 8B ...
Attributes:
    AREA_OR_POINT:  Area

The "x" and "y" coordinates with a raster index are lazy! The repr below shows a few values for each coordinate (those have been computed on-the-fly) but clicking on the database icon doesn't show any value in the spatial coordinate data reprs.

In [6]:
da_raster_index

<xarray.DataArray 'band_data' (band: 1, y: 332, x: 316)> Size: 420kB
[104912 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B ...
  * x            (x) float64 3kB -3.938e+06 -3.912e+06 ... 3.912e+06 3.938e+06
  * y            (y) float64 3kB 4.338e+06 4.312e+06 ... -3.912e+06 -3.938e+06
Indexes:
  ┌ x        RasterIndex
  └ y
Attributes:
    AREA_OR_POINT:  Area

In [7]:
type(da_raster_index.coords.variables["x"]._data)

xarray.core.indexing.CoordinateTransformIndexingAdapter

### Compare and align the datasets with and without `RasterIndex`

`equals` compares variable values without relying on Xarray coordinate indexes. Both dataarrays should thus be equal.

In [8]:
da_raster_index.equals(da_no_raster_index)

True

Xarray alignment relies on Xarray coordinate indexes. Trying to align both datasets fails here since they each have different index types.

Maybe Xarray should try aligning the datasets based on coordinate variable data in this case? I don't think this would be easy to implement in a general way... Xarray's alignment logic is already complex! Also the alignment failure here is not necessarily a bad thing (cf. alignment issues with explicit floating-point coordinates).

In [9]:
# this fails!

# da_raster_index + da_no_raster_index

### Indexing the dataarray with `RasterIndex`

#### Integer-based selection (isel)

- *Slicing both x and y*

In [10]:
da_sliced = da_raster_index.isel(x=slice(1, 4), y=slice(None, None, 2))
da_sliced

<xarray.DataArray 'band_data' (band: 1, y: 166, x: 3)> Size: 2kB
array([[[0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.]]], shape=(1, 166, 3), dtype=float32)
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B 0
  * x            (x) float64 24B -3.912e+06 -3.888e+06 -3.862e+06
  * y            (y) float64 1kB 4.338e+06 4.288e+06 ... -3.862e+06 -3.912e+06
Indexes:
  ┌ x        RasterIndex
  └ y
Attributes:
    AREA_OR_POINT:  Area

Slicing keeps both coordinates lazy (it computes a new affine transform):

In [11]:
print(da_sliced.xindexes["x"])
print(da_sliced.xindexes["y"])

RasterIndex
'x':
'y':
RasterIndex
'x':
'y':


- *Outer indexing with arbitrary array values*

In [12]:
da_outer = da_raster_index.isel(x=[0, 2, 4], y=[0, 0, 1])
da_outer

<xarray.DataArray 'band_data' (band: 1, y: 3, x: 3)> Size: 36B
array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]], dtype=float32)
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B 0
  * x            (x) float64 24B -3.938e+06 -3.888e+06 -3.838e+06
  * y            (y) float64 24B 4.338e+06 4.338e+06 4.312e+06
Indexes:
  ┌ x        RasterIndex
  └ y
Attributes:
    AREA_OR_POINT:  Area

We cannot compute a new affine transform given arbitrary array positions. To allow further data selection, pandas indexes are created for indexed spatial coordinates:

In [13]:
print(da_outer.xindexes["x"])
print(da_outer.xindexes["y"])

RasterIndex
'x':
    PandasIndex(Index([-3937500.0, -3887500.0, -3837500.0], dtype='float64', name='x'))
'y':
    PandasIndex(Index([4337500.0, 4337500.0, 4312500.0], dtype='float64', name='y'))
RasterIndex
'x':
    PandasIndex(Index([-3937500.0, -3887500.0, -3837500.0], dtype='float64', name='x'))
'y':
    PandasIndex(Index([4337500.0, 4337500.0, 4312500.0], dtype='float64', name='y'))


- *Basic indexing with scalars*

In [14]:
da_scalar = da_raster_index.isel(x=0, y=1)
da_scalar

<xarray.DataArray 'band_data' (band: 1)> Size: 4B
array([0.], dtype=float32)
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B 0
    x            float64 8B -3.938e+06
    y            float64 8B 4.312e+06
Attributes:
    AREA_OR_POINT:  Area

In [15]:
da_xscalar = da_raster_index.isel(x=0)
da_xscalar

<xarray.DataArray 'band_data' (band: 1, y: 332)> Size: 1kB
array([[0., 0., 0., ..., 0., 0., 0.]], shape=(1, 332), dtype=float32)
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B 0
    x            float64 8B -3.938e+06
    y            (y) float64 3kB 4.338e+06 4.312e+06 ... -3.912e+06 -3.938e+06
Attributes:
    AREA_OR_POINT:  Area

**FIXME** The RasterIndex should be preserved in case of partial dimension reduction.

In [16]:
# da_xscalar.xindexes["y"]  # should return an index

- *Vectorized (fancy) indexing*

Indexing the spatial coordinates with Xarray `Variable` objects returns a `RasterIndex` (wrapping `PandasIndex`) for 1-dimensional variables and no index for scalar or n-dimensional variables.

In [17]:
da_points = da_raster_index.isel(x=xr.Variable("z", [0, 1]), y=xr.Variable("z", [1, 1]))
da_points

<xarray.DataArray 'band_data' (band: 1, z: 2)> Size: 8B
array([[0., 0.]], dtype=float32)
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B 0
  * x            (z) float64 16B -3.938e+06 -3.912e+06
  * y            (z) float64 16B 4.312e+06 4.312e+06
Dimensions without coordinates: z
Indexes:
  ┌ x        RasterIndex
  └ y
Attributes:
    AREA_OR_POINT:  Area

In [18]:
da_points2d = da_raster_index.isel(
    x=xr.Variable(("u", "v"), [[0, 1], [2, 3]]),
    y=xr.Variable(("u", "v"), [[1, 1], [2, 2]]),
)
da_points2d

<xarray.DataArray 'band_data' (band: 1, u: 2, v: 2)> Size: 16B
array([[[0., 0.],
        [0., 0.]]], dtype=float32)
Coordinates:
  * band         (band) int64 8B 1
    spatial_ref  int64 8B 0
    x            (u, v) float64 32B -3.938e+06 -3.912e+06 -3.888e+06 -3.862e+06
    y            (u, v) float64 32B 4.312e+06 4.312e+06 4.288e+06 4.288e+06
Dimensions without coordinates: u, v
Attributes:
    AREA_OR_POINT:  Area

#### Label-based selection (sel)

TODO

## Example with complex affine transformation

x and y coordinates are both 2-dimensional.

TODO